In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras import metrics
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.layers import Flatten

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

C:\Users\geop\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [101]:
#Load cleaned dataset
data = pd.read_csv('../../Results/25_cleaned_job_descriptions.csv', header = 0, names = ['Query', 'Description'])

In [102]:
#Split data to train and test (80 - 20)
train, test = train_test_split(data, test_size = 0.2)

train_descs = train['Description']
train_labels = train['Query']
#train_labels = train['Job Title']
 
test_descs = test['Description']
test_labels = test['Query']
#test_labels = test['Job Title']

In [103]:
##Parameters
#Encoding
vocab_size = 500
max_length = 500

#Model
num_labels = 25
embedding_dimensios = 20
nb_epoch = 30
batch_size = 100

In [104]:
###Training Data
# Encode the jobs descriptions
encoded_docs = [one_hot(d, vocab_size) for d in train_descs]
# pad documents to a max length
x_train = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#Binarize the job titles
encoder = LabelBinarizer()
encoder.fit(train_labels)
y_train = encoder.transform(train_labels)


###Test Data
# Encode the jobs descriptions
encoded_docs = [one_hot(d, vocab_size) for d in test_descs]
# pad documents to a max length
x_test = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#Binarize the job titles
encoder = LabelBinarizer()
encoder.fit(test_labels)
y_test = encoder.transform(test_labels)

In [105]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dimensios, input_length=max_length))
model.add(Flatten())
model.add(Dense(num_labels))
model.add(Activation('softmax'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 20)           10000     
_________________________________________________________________
flatten_2 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 25)                250025    
_________________________________________________________________
activation_2 (Activation)    (None, 25)                0         
Total params: 260,025
Trainable params: 260,025
Non-trainable params: 0
_________________________________________________________________
None


In [106]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam', # or 'sgd'
              metrics = [metrics.categorical_accuracy, 'accuracy'])

In [108]:
# fit the model
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_split=0.1)

Train on 7200 samples, validate on 800 samples
Epoch 1/30
7200/7200 [==============================] - 1s 89us/step - loss: 0.0569 - categorical_accuracy: 0.9908 - acc: 0.9908 - val_loss: 2.5973 - val_categorical_accuracy: 0.3575 - val_acc: 0.3575
Epoch 2/30
7200/7200 [==============================] - 1s 88us/step - loss: 0.0535 - categorical_accuracy: 0.9918 - acc: 0.9918 - val_loss: 2.5944 - val_categorical_accuracy: 0.3550 - val_acc: 0.3550
Epoch 3/30
7200/7200 [==============================] - 1s 87us/step - loss: 0.0558 - categorical_accuracy: 0.9903 - acc: 0.9903 - val_loss: 2.6108 - val_categorical_accuracy: 0.3575 - val_acc: 0.3575
Epoch 4/30
7200/7200 [==============================] - 1s 89us/step - loss: 0.0609 - categorical_accuracy: 0.9907 - acc: 0.9907 - val_loss: 2.6084 - val_categorical_accuracy: 0.3575 - val_acc: 0.3575
Epoch 5/30
7200/7200 [==============================] - 1s 89us/step - loss: 0.0557 - categorical_accuracy: 0.9907 - acc: 0.9907 - val_loss: 2.6238 -

In [109]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('\nTest categorical_cro    ssentropy:', score[0])
print('Categorical accuracy:', score[1])
print('Accuracy:', score[2])

2000/2000 [==============================] - 0s 30us/step

Test categorical_cro    ssentropy: 2.924647796154022
Categorical accuracy: 0.3425000004470348
Accuracy: 0.3425000004470348


In [114]:
#Prediction Function

def Prediction(model,user_text):
    
    # Encode the text
    encoded_docs = [one_hot(user_text, vocab_size)]
    
    # pad documents to a max length
    padded_text = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    
    #Prediction based on model
    prediction = model.predict(padded_text)
    
    #Decode the prediction
    encoder = LabelBinarizer()
    encoder.fit(test_labels)
    result = encoder.inverse_transform(prediction)
    
    print("I think the best job for you is", result[0])
    
   

In [117]:
Prediction(model,'data')

I think the best job for you is Data Analyst
